<h1>
<center>CFRM 521, Spring 2020</center>
</h1>

<h1>
<center>[Please insert your name(s) here]</center>
</h1>

<h1>
<center>Homework 1</center>
</h1>

* **Due: Monday 20th April 2020, at 11:59pm**


* You may submitted this homework with one other classmate. If you are submitting in a group of two, only one of you must submit the homework. The other must make a comment on Canvas (where you would submit the homework) mentioning that they are submitting with another person. Also, please include both of your names in the homework submission.


* Late homework are allowed, **but a 10% penalty per day applies.** Example: submitting on Tuesday 00:01am results in 10% penalty, on Wednesday 00:01am results in 20% penalty, etc. Your last submission is considered for calculating the penalty.


*  Please use this Jupyter notebook as a template for your solutions. Your solution must be submitted as one Jupyter notebook. You are allowed to use code from the textbook, textbook website, or lecture notes. However, please go over the code you are using.

# 1. Preprocessing Housing data

Consider the California housing data from Chapter 2 of the textbook. Let us fetch the data (see Lecture 1 notes for explanation of the following code).

In [61]:
import os
import tarfile
from six.moves import urllib
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

HOUSING_PATH = os.path.join("datasets", "housing")

def fetch_housing_data(housing_url, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

HOUSING_URL = ("https://raw.githubusercontent.com/ageron/"+
               "handson-ml2/master/datasets/housing/housing.tgz")
fetch_housing_data(HOUSING_URL)
data = load_housing_data()

Next, we create stratified test and training sets based on different income categories (see lecture one notes for explanation of the code).

In [62]:
data["income_cat"] = np.ceil(data["median_income"] / 1.5)
data["income_cat"].where(data["income_cat"] < 5, 5.0, inplace=True)

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(data, data["income_cat"]):
    strat_train_set = data.loc[train_index]
    strat_test_set = data.loc[test_index]
    
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

Let us separate features (`X_raw`) and the response variable (`y`).

In [63]:
X_raw = strat_train_set.drop("median_house_value", axis=1)
y = strat_train_set["median_house_value"].copy()

## (a) Handling missing values

Let us find out if there is any missing value in our features. As the code below indicates, out of 16512 observations, 158 have missing values. Further inspection shows that `total_bedrooms` has missing values (only a few missing values is shown).

In [64]:
X_raw.shape

(16512, 9)

In [65]:
X_raw[X_raw.isnull().any(axis=1)].shape

(158, 9)

In [66]:
X_raw[X_raw.isnull().any(axis=1)].head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
4629,-118.30,34.07,18.0,3759.0,NaN,3296.0,1462.0,2.2708,<1H OCEAN
6068,-117.86,34.01,16.0,4632.0,NaN,3038.0,727.0,5.1762,<1H OCEAN
17923,-121.97,37.35,30.0,1955.0,NaN,999.0,386.0,4.6328,<1H OCEAN
13656,-117.30,34.05,6.0,2155.0,NaN,1039.0,391.0,1.6675,INLAND
19252,-122.79,38.48,7.0,6837.0,NaN,3468.0,1405.0,3.1662,<1H OCEAN


**Task:** Use the `sklearn.impute.SimpleImputer` class to fill the missing values in the numerical features (everything except `ocean_proximity`) with the median of the corresponding feature.

Hint: In Chapter 2 of the textbook, under section "Prepare the Data for Machine Learning Algorithms", you can find explanation how to fill missing values. Also, take a look at the corresponding Jupyter notebook from the textbook website.

**[Add your solution here]**

In [67]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")
X_raw_num = X_raw.drop("ocean_proximity", axis=1)
imputer.fit(X_raw_num)
S = imputer.transform(X_raw_num)
X_tr = pd.DataFrame(S, columns=X_raw_num.columns,index=X_raw_num.index)
X_tr.shape

(16512, 8)

## (b) Handling categorical features
ML algorithms generally work with numerical values. So, we need to convert categorical (or text) features to numerical values. One way is to simply map each category to an integer. Another alternative is to use **one-hot encoding**.

**Task:** Read subsection "Handling Text and Categorical Attributes" in Chapter 2 of the textbook to learn one-hot encoding and its advantages. Then use the `sklearn.preprocessing.OrdinalEncoder` class and the `sklearn.preprocessing.OneHotEncoder` class to transform the `ocean_proximity` feature to integers and one-hot vectors.

**[Add your solution here]**

In [68]:
from sklearn.preprocessing import OrdinalEncoder
X_cat = X_raw[["ocean_proximity"]]
ordinal_encoder = OrdinalEncoder()
X_cat_encoded = ordinal_encoder.fit_transform(X_cat)
X_cat_encoded[:10]

array([[0.],
       [0.],
       [4.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.]])

In [69]:
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()
X_cat_1hot = cat_encoder.fit_transform(X_cat)
X_cat_1hot

<16512x5 sparse matrix of type '<class 'numpy.float64'>'
	with 16512 stored elements in Compressed Sparse Row format>

## (c) Feature scaling

Many ML algorithms do not perform well if the features have different scales (some very small, and others very large). In such scenarios, one should transform the features so that they have similar range of values. There are two common ways to achieve this. In **min-max** scaling, we subtract the feature from its minimum value and then divide by its range (i.e. maximum value minus minimum value) so that the scaled values are between 0 and 1. In **standardization**, we subtract the values from the average and divide by the standard deviation, so that the transformed values has mean 0 and variance 1.

**Task:** Use the `sklearn.preprocessing.MinMaxScaler` class and `sklearn.preprocessing.StandardScaler` class to scale the numerical features using min-max scaling and standardization.


**[Add your solution here]**

In [70]:
from sklearn.preprocessing import MinMaxScaler
minmax_scaler=MinMaxScaler()
X_minmax=minmax_scaler.fit_transform(X_tr)
X_minmax

array([[0.24501992, 0.50478215, 0.7254902 , ..., 0.01981558, 0.06292009,
        0.15201859],
       [0.24103586, 0.47927736, 0.25490196, ..., 0.00849239, 0.02072442,
        0.40837368],
       [0.71215139, 0.02444208, 0.58823529, ..., 0.02614984, 0.08588499,
        0.1629081 ],
       ...,
       [0.79183267, 0.16471838, 0.15686275, ..., 0.05871801, 0.14245706,
        0.19119736],
       [0.6314741 , 0.1360255 , 0.58823529, ..., 0.03792147, 0.0660941 ,
        0.24569316],
       [0.18924303, 0.55579171, 1.        , ..., 0.03548306, 0.11893204,
        0.21207294]])

In [71]:
from sklearn.preprocessing import StandardScaler
standard_scaler=StandardScaler()
X_std=standard_scaler.fit_transform(X_tr)
X_std

array([[-1.15604281,  0.77194962,  0.74333089, ..., -0.63621141,
        -0.42069842, -0.61493744],
       [-1.17602483,  0.6596948 , -1.1653172 , ..., -0.99833135,
        -1.02222705,  1.33645936],
       [ 1.18684903, -1.34218285,  0.18664186, ..., -0.43363936,
        -0.0933178 , -0.5320456 ],
       ...,
       [ 1.58648943, -0.72478134, -1.56295222, ...,  0.60790363,
         0.71315642, -0.3167053 ],
       [ 0.78221312, -0.85106801,  0.18664186, ..., -0.05717804,
        -0.37545069,  0.09812139],
       [-1.43579109,  0.99645926,  1.85670895, ..., -0.13515931,
         0.3777909 , -0.15779865]])

## (d) Transformation pipelines

Usually, we have to perform several steps before the data is ready to be fed to an ML algorithm. Scikit-Learn's `Pipeline` class provides a systematic way of "packaging" such sequence of transformations.

**Task:** Read the subsection titled "transformation pipelines" in Chapter 2 of the textbook. Then, create one pipeline that applies the transformations on part (a), (b) (use one-hot encoding), and (c) (use standardization) to the "raw" features `X_raw`. Call your transformed data `X`.

**[Add your solution here]**

In [72]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

num_pipeline = Pipeline([
('imputer', SimpleImputer(strategy="median")),
('std_scaler', StandardScaler()),
])

num_attribs = list(X_raw_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
("num", num_pipeline, num_attribs),
("cat", OneHotEncoder(), cat_attribs),
])


X = full_pipeline.fit_transform(X_raw)
X

array([[-1.15604281,  0.77194962,  0.74333089, ...,  0.        ,
         0.        ,  0.        ],
       [-1.17602483,  0.6596948 , -1.1653172 , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.18684903, -1.34218285,  0.18664186, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 1.58648943, -0.72478134, -1.56295222, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.78221312, -0.85106801,  0.18664186, ...,  0.        ,
         0.        ,  0.        ],
       [-1.43579109,  0.99645926,  1.85670895, ...,  0.        ,
         1.        ,  0.        ]])

# 2 Fine-tuning models

## (a) Linear regression

**Task:** Use the `sklearn.linear_model.LinearRegression` class to fit a multiple linear regression to the training set in Question 1. Use the processed features `X` that you obtained in 1.(d) as predictors and `y` (the median house values) as the response. Return fitted values of the response for the first 10 observations of the training set.

**[Add your solution here]**

In [73]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X, y)
some_data = X[:10]
some_labels = y.iloc[:10]
print("Labels:", list(some_labels))
print("Predict:", list(lin_reg.predict(some_data)))

Labels: [286600.0, 340600.0, 196900.0, 46300.0, 254500.0, 127900.0, 500001.0, 140200.0, 95000.0, 500001.0]
Predict: [211574.39523832555, 321345.1051371902, 210947.51983800187, 61921.01197837471, 192362.3296111861, 154821.86099030558, 426129.59146333823, 229666.36167750446, 141483.54356475634, 12084.195020017622]


## (b) RMSE and MAE

As we have seen in CFRM 502, we can measure the out-of-sample performance with root mean square error (RMSE) and mean absolute error (MAE). 

**Task:** Use `mean_squared_error` and `mean_absolute_error` functions from `sklearn.metrics` to calculate the in-sample RMSE and MAE of the linear regressor that you fit in part (a).

**[Add your solution here]**

In [74]:
from sklearn.metrics import mean_squared_error
y_predictions = lin_reg.predict(X)
lin_mse = mean_squared_error(y, y_predictions)
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

from sklearn.metrics import mean_absolute_error
lin_mae = mean_absolute_error(y, y_predictions)
print(lin_mae)

69050.98178244587
49906.94142223288


## (c) Cross validation

In part (b), we evaluated the in-sample performance of the linear regression model. To obtain out-of-sample performance, we may use K-fold cross validation as was explained in Lecture 2.

**Task:** Use the `cross_val_score` function from `sklearn.model_selection` to perform 20-fold  cross validation on the linear regressor from part (a) and return the values of **MAE** (use the argument `scoring="neg_mean_absolute_error"`). Return the 20 MAE scores as well as their mean and standard deviations.

**[Add your solution here]**

In [75]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(lin_reg, X, y,scoring="neg_mean_absolute_error", cv=20)
lin_mae_scores = (-scores)

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())
    
display_scores(lin_mae_scores)

Scores: [52128.74309417 45734.66433295 50755.22456793 47840.45345004
 49264.3935965  49998.72004939 49848.56169884 53690.92931375
 52060.21094438 48484.83022226 52802.79659221 50376.81642374
 47184.38509378 49163.95427331 47702.70481427 50525.42113931
 51272.31717921 51684.20719774 50591.06918414 48426.96686148]
Mean: 49976.86850146998
Standard deviation: 1976.5152666639403


## (d) Alternatives to linear regression
Let us consider fitting two alternative models, namely, **decision trees** and **random forests**. These models will be discussed during future lectures. The following code fits a decision tree and obtain the fitted response for the first 10 observations.

In [76]:
from sklearn.tree import DecisionTreeRegressor
dt_reg = DecisionTreeRegressor(random_state=42)
dt_reg.fit(X,y)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=42, splitter='best')

In [77]:
dt_reg.predict(X[:10,])

array([286600., 340600., 196900.,  46300., 254500., 127900., 500001.,
       140200.,  95000., 500001.])

The following code do the same things for the random forest model.

In [78]:
from sklearn.ensemble import RandomForestRegressor

rf_reg = RandomForestRegressor(bootstrap=True, n_estimators=10,
                                max_features=2, random_state=42)
rf_reg.fit(X, y)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features=2, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=10, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [79]:
rf_reg.predict(X[:10,])

array([252210. , 350830.1, 202920. ,  52360. , 241080. , 114640. ,
       466600.6, 202360. ,  94890. , 384110.7])

**Task:** Find the in-sample performance of the decision tree and the random forest using MAE on the training set. Also, find the out-of-sample performance by the 20-fold cross validation as you did in part (c). Interpret the results. Which model has the best in-sample performance? Which model has the best out-of-sample performance?

**[Add your solution here]**

In [80]:
dt_mae = mean_absolute_error(y, dt_reg.predict(X))
print("dt_mae:",dt_mae)

dt_scores = cross_val_score(dt_reg, X, y,scoring="neg_mean_absolute_error", cv=20)
dt_mae_scores = (-dt_scores)
display_scores(dt_mae_scores)

rf_mae = mean_absolute_error(y, rf_reg.predict(X))
print("rf_mae:",rf_mae)

rf_scores = cross_val_score(rf_reg, X, y,scoring="neg_mean_absolute_error", cv=20)
rf_mae_scores = (-rf_scores)
display_scores(rf_mae_scores)

dt_mae: 0.0
Scores: [45750.65133172 38632.7094431  43592.06053269 44728.73486683
 42278.7251816  46432.2433414  42164.68038741 47100.65012107
 44650.40556901 42894.47215496 47275.23244552 47095.44430993
 47099.79757576 44367.56242424 43305.13454545 43804.40242424
 43356.04727273 48016.77939394 44114.70545455 42722.96      ]
Mean: 44469.169938806954
Standard deviation: 2240.992398053244
rf_mae: 14797.653906250001
Scores: [39384.00072639 33155.38280872 35883.43075061 37209.92397094
 36649.50992736 38476.97711864 35869.85423729 38949.37336562
 37980.79358354 35885.24939467 39133.39249395 38779.46549637
 36704.21418182 38004.79672727 34175.1169697  39735.49551515
 38031.02848485 38771.17539394 38312.22581818 35254.19151515]
Mean: 37317.27992400763
Standard deviation: 1768.8407619514485


## (e) Choosing good values of hyperparameters using Cross Validation

Most ML algorithms have **hyperparameters**. These hyperparameters control the learning process and should not be confused with the model parameters (which are the unknown values that are being estimated). For example, the random forest algorithm has 3 hyperparameters, namely `bootstrap`, `n_estimators`, and `max_features`. To fine-tune a model, we should find good values of hyperparameters based on out-of-sample performance.

**Task:** Read sections titled "Grid Search" and "Randomized Search" in Chapter 2 of the textbook. Then, use the `sklearn.model_selection.GridSearchCV` class and `sklearn.model_selection.RandomizedSearchCV` class to choose a good set of values of the `bootstrap`, `n_estimators`, and `max_features` hyperparamters of the random forest algorithm.  Use 5-fold cross-validation when searching for the hyper parameters. But, report MAE values for the 20-fold cross validation of the best model you find. Also, report the best values of the hyperparameters.

**[Add your solution here]**

In [81]:
from sklearn.model_selection import GridSearchCV
param_grid = [{'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},{'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},]
forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,scoring='neg_mean_squared_error',return_train_score=True)
grid_search.fit(X, y)

grid_search.best_estimator_

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features=8, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=30, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [82]:
rf_reg = RandomForestRegressor(bootstrap=True, n_estimators=30,
                                max_features=8, random_state=42)
rf_reg.fit(X, y)
rf_scores = cross_val_score(rf_reg, X, y,scoring="neg_mean_absolute_error", cv=20)
rf_mae_scores =(-rf_scores)
display_scores(rf_mae_scores)

Scores: [35645.4693301  28416.21299435 30322.82215496 31230.87655367
 31334.00221953 32894.58684423 30600.93910412 34415.14717514
 33855.72675545 31017.7559322  33906.72683616 34813.29834544
 32355.98525253 32726.6680404  30850.23866667 33241.50949495
 32944.17046465 34553.4640404  33067.24040404 31013.82521212]
Mean: 32460.333291055846
Standard deviation: 1779.5152662616097


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt

parameters = {'n_estimators' : sp_randInt(1, 50), 'max_features': sp_randInt(1, 10)}
randm = RandomizedSearchCV(forest_reg, parameters,n_iter = 100, scoring="neg_mean_absolute_error",cv = 5)
randm.fit(X, y)
randm.best_estimator_

In [ ]:
rf_reg = RandomForestRegressor(bootstrap=True, n_estimators=41,
                                max_features=8, random_state=42)
rf_reg.fit(X, y)

rf_scores = cross_val_score(rf_reg, X, y,scoring="neg_mean_absolute_error", cv=20)
rf_mae_scores =(-rf_scores)
display_scores(rf_mae_scores)

## (f) Evaluating your final model using the test set

The last step of a ML project is to evaluate your (fine-tuned) model using the test set. The goal of this step is to estimate the out-of-sample performance of your trained model on new data. **Be careful not to train your model using the test set!** Do not use `.fit()` or `.fit_tranform()` of your estimators or pipelines. Also, **do not fine tune your hyperparameters after finding out the performance of a model on the test set!** Doing so is data snooping.

**Task:** Report the MAE and RMSE of your best model in part (e). Compare with the average scores of the 20-fold cross validation using the training set. Are the test scores better than the average of the cross validation scores?

**[Add your solution here]**

In [ ]:
test_raw = strat_test_set.drop("median_house_value", axis=1)
test_p = strat_test_set["median_house_value"].copy()

testing = full_pipeline.transform(test_raw)

test_predictions = rf_reg.predict(testing)

test_mse = mean_squared_error(test_p, test_predictions)
test_rmse = np.sqrt(test_mse)
print(test_rmse)

test_mae = mean_absolute_error(test_p, test_predictions)
print(test_mae)

#The test score is not better than the average of the cross validation scores.

# 3. Regularizing linear regression

Consider the housing data set from Questions 1 and 2.

## (a) learning curves

Obtain learning curves for the linear regression model that you fitted in Question 2.(a). To make your code faster, do not add one observation at at time to obtain the learning curves. Instead, add, say, 100 observations to obtain subsequent points of the curves. This way, you will end up fitting fewer models (and spend less time). Do the curves indicate over-fitting or under-fitting?

**[Add your solution here]**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
def plot_learning_curves(lin_reg, X, y):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
    train_errors, val_errors = [], []
    for m in range(100, len(X_train)):
        lin_reg.fit(X_train[:m], y_train[:m])
        y_train_predict = lin_reg.predict(X_train[:m])
        y_val_predict = lin_reg.predict(X_val)
        train_errors.append(mean_squared_error(y_train[:m], y_train_predict))
        val_errors.append(mean_squared_error(y_val, y_val_predict))
    plt.plot(np.sqrt(train_errors), "r-+", linewidth=2, label="train")
    plt.plot(np.sqrt(val_errors), "b-", linewidth=3, label="val")

In [ ]:
reg = LinearRegression()
plot_learning_curves(reg, X, y)

#It indicates a underfitting curve. Performance on training data is worse than on testing set. And they are both pretty high.

## (b) Regularizing

If you think your model is over-fitting, try to regularize it with ridge, lasso, or elastic net (one model is sufficient). If the model is under-fitting, add enough polynomial features so that it over-fits the training set, and then regularize it. Find a good value of the regularization hyperparameter(s) by grid search as you did in Question 2.(e) of Homework 1. Report the learning curves of the best model you find.

**[Add your solution here]**

In [ ]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=1, solver="cholesky")
ridge_reg.fit(X, y)

plot_learning_curves(ridge_reg, X, y)

# 4. Titanic dataset

In this question you will deal with the Titanic dataset, which is a nice "Tutorial" example for ML. Take a look at the following page on Kaggle for further information [https://www.kaggle.com/c/titanic/](https://www.kaggle.com/c/titanic/) (you need to register). You can also use the solution of Exercise 3 of Chapter 3 of the textbook which is included in the online Jupyter notebook from [https://github.com/ageron/handson-ml2](https://github.com/ageron/handson-ml2). As always, please experiment with the code instead of just copy-pasting!

## (a)
Load the data and split it into training and test set. Note that you cannot use the `test.csv` from Kaggle, since it does not have the labels (the column `Survived`). You have to create your own traning and test sets using the `train.csv` (use 20% of the data for testing). Provide a brief description of the features, the label, and summary statistics.

**[Add your solution here]**

In [ ]:
import pandas as pd

data= pd.read_csv("train.csv")
data
#For some reasons, I keep getting error when loading data. And it also happens when running the github textbook solution. So I used downloaded version.

from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(data, test_size=0.2, random_state=42)

data.info()
data.describe()

In [ ]:
data.drop("PassengerId", axis=1, inplace=True)
data.drop("Name", axis=1, inplace=True)
data.drop("Cabin", axis=1, inplace=True)
data.drop("Ticket", axis=1, inplace=True)

%matplotlib inline
import matplotlib.pyplot as plt
scale = 2.75
data.hist(bins=50, figsize=(5*scale,2*scale), layout=(2,5))
plt.show()

## (b)
Design a pipeline for preprocessing the features. Explain your steps and why you are taking them.

In [ ]:
X_raw = data.drop("Survived", axis=1)
y = data["Survived"].copy()

**[Add your solution here]**

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin

X_num = X_raw.drop(["Sex","Embarked","Pclass"], axis=1)
X_cat = X_raw[["Sex","Embarked","Pclass"]]

class MostFrequentImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.most_frequent_ = pd.Series([X[c].value_counts().index[0] for c in X],
                                        index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.most_frequent_)

In [ ]:
num_pipeline = Pipeline([
('imputer', SimpleImputer(strategy="median")),
('std_scaler', StandardScaler()),
])


cat_pipeline = Pipeline([
('imputer', MostFrequentImputer()),
('1hotencode', OneHotEncoder()),
])


num_attribs = list(X_num)
cat_attribs = list(X_cat)

full_pipeline = ColumnTransformer([
("num", num_pipeline, num_attribs),
("cat", cat_pipeline, cat_attribs),
])


X = full_pipeline.fit_transform(X_raw)
X

## (c)
Train a logistic regression classifier and a `SGDClassifier` (a linear support vector machine trained with SGD).

**[Add your solution here]**

In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(X, y)

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X, y)

## (d)
For the classifiers in part (c), plot precision against recall for various decision thresholds. What do you think is more important in this classification task, decision or recall?

**[Add your solution here]**

In [ ]:
from sklearn.model_selection import cross_val_predict
y_scores = cross_val_predict(log_reg, X, y, cv=3,
method="decision_function")

from sklearn.metrics import precision_recall_curve
precisions, recalls, thresholds = precision_recall_curve(y, y_scores)

def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision")
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall")

plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.show()

In [ ]:
from sklearn.model_selection import cross_val_predict
y_scores = cross_val_predict(sgd_clf, X, y, cv=3,
method="decision_function")

from sklearn.metrics import precision_recall_curve
precisions, recalls, thresholds = precision_recall_curve(y, y_scores)

def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision")
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall")

plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.show()

#recall is more important here for us to classify if passengers survided or not.

## (e)
Plot ROC curve of the classifiers in part (c) and compute their ROC-AUC. Perform, also, 10-fold cross validation and report accuracy. Based on your findings, which classifier is better?

**[Add your solution here]**

In [ ]:
from sklearn.model_selection import cross_val_predict
y_scores = cross_val_predict(log_reg, X, y, cv=3,
method="decision_function")

from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y, y_scores)

def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--') # Dashed diagonal
plot_roc_curve(fpr, tpr)
plt.show()

In [ ]:
from sklearn.model_selection import cross_val_predict
y_scores = cross_val_predict(sgd_clf, X, y, cv=3,
method="decision_function")

from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y, y_scores)

def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--') # Dashed diagonal
plot_roc_curve(fpr, tpr)
plt.show()


In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y, y_scores)

#logistic model is better, its ROC is closer to 1.

## (f)

Build a classifier that has over 80% average accuracy using 10-fold cross validation over the training set. Train the classifier over the whole training set and report its accuracy, precision, recall, ROC curve and ROC-AUC over the test set.

**[Add your solution here]**